In [1]:
%cd /kaggle/working
!git clone https://github.com/ultralytics/yolov5  # clone repo
%cd yolov5

/kaggle/working
Cloning into 'yolov5'...
remote: Enumerating objects: 16614, done.
remote: Counting objects: 100% (92/92), done.
remote: Compressing objects: 100% (65/65), done.
remote: Total 16614 (delta 51), reused 59 (delta 27), pack-reused 16522
Receiving objects: 100% (16614/16614), 15.15 MiB | 27.32 MiB/s, done.
Resolving deltas: 100% (11408/11408), done.
/kaggle/working/yolov5


In [4]:
%pip install -qr requirements.txt  # install dependencies (ignore errors)
import torch

from IPython.display import Image, clear_output  # to display images
from utils.downloads import attempt_download  # to download models/datasets

ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
spopt 0.6.0 requires shapely>=2.0.1, but you have shapely 1.8.5.post1 which is incompatible.
ydata-profiling 4.6.4 requires numpy<1.26,>=1.16.0, but you have numpy 1.26.4 which is incompatible.
Note: you may need to restart the kernel to use updated packages.


In [6]:
!pip install roboflow

from roboflow import Roboflow
rf = Roboflow(api_key="1gBpoxVirzj0NHBuskuN")
project = rf.workspace("capstone-design-2").project("fire-smoke-7rocs")
version = project.version(1)
dataset = version.download("yolov5")

loading Roboflow workspace...
loading Roboflow project...


In [7]:
%cd /kaggle/working/yolov5

/kaggle/working/yolov5


In [11]:
import yaml
with open(dataset.location + "/data.yaml", 'r') as stream:
    num_classes = str(yaml.safe_load(stream)['nc'])

In [14]:
#customize iPython writefile so we can write variables
from IPython.core.magic import register_line_cell_magic

@register_line_cell_magic
def writetemplate(line, cell):
    with open(line, 'w') as f:
        f.write(cell.format(**globals()))

In [16]:
%%writetemplate /kaggle/working/yolov5/models/custom_yolov5s.yaml

# parameters
nc: {num_classes}  # number of classes
depth_multiple: 0.33  # model depth multiple
width_multiple: 0.50  # layer channel multiple

# anchors
anchors:
  - [10,13, 16,30, 33,23]  # P3/8
  - [30,61, 62,45, 59,119]  # P4/16
  - [116,90, 156,198, 373,326]  # P5/32

# YOLOv5 backbone
backbone:
  # [from, number, module, args]
  [[-1, 1, Focus, [64, 3]],  # 0-P1/2
   [-1, 1, Conv, [128, 3, 2]],  # 1-P2/4
   [-1, 3, BottleneckCSP, [128]],
   [-1, 1, Conv, [256, 3, 2]],  # 3-P3/8
   [-1, 9, BottleneckCSP, [256]],
   [-1, 1, Conv, [512, 3, 2]],  # 5-P4/16
   [-1, 9, BottleneckCSP, [512]],
   [-1, 1, Conv, [1024, 3, 2]],  # 7-P5/32
   [-1, 1, SPP, [1024, [5, 9, 13]]],
   [-1, 3, BottleneckCSP, [1024, False]],  # 9
  ]
# YOLOv5 head
head:
  [[-1, 1, Conv, [512, 1, 1]],
   [-1, 1, nn.Upsample, [None, 2, 'nearest']],
   [[-1, 6], 1, Concat, [1]],  # cat backbone P4
   [-1, 3, BottleneckCSP, [512, False]],  # 13

   [-1, 1, Conv, [256, 1, 1]],
   [-1, 1, nn.Upsample, [None, 2, 'nearest']],
   [[-1, 4], 1, Concat, [1]],  # cat backbone P3
   [-1, 3, BottleneckCSP, [256, False]],  # 17 (P3/8-small)

   [-1, 1, Conv, [256, 3, 2]],
   [[-1, 14], 1, Concat, [1]],  # cat head P4
   [-1, 3, BottleneckCSP, [512, False]],  # 20 (P4/16-medium)

   [-1, 1, Conv, [512, 3, 2]],
   [[-1, 10], 1, Concat, [1]],  # cat head P5
   [-1, 3, BottleneckCSP, [1024, False]],  # 23 (P5/32-large)

   [[17, 20, 23], 1, Detect, [nc, anchors]],  # Detect(P3, P4, P5)
  ]

In [19]:
%%time
%cd /kaggle/working/yolov5/
!python train.py --img 640 --batch 128 --epochs 20 --device 0,1 --data {dataset.location}/data.yaml --cfg ./models/custom_yolov5s.yaml --weights '' --name yolov5s_results --resume true --cache

/kaggle/working/yolov5
wandb: WARNING ⚠️ wandb is deprecated and will be removed in a future release. See supported integrations at https://github.com/ultralytics/yolov5#integrations.
2024-05-16 10:03:04.963102: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-05-16 10:03:04.963163: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-05-16 10:03:04.964701: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
wandb: (1) Create a W&B account
wandb: (2) Use an existing W&B account
wandb: (3) Don't visualize my results
wandb: Enter your choice: (30 second timeout) 
wandb: W&B disabled due t

In [22]:
!python export.py --weights /kaggle/working/yolov5/runs/train/yolov5s_results/weights/last.pt --include onnx

export: data=data/coco128.yaml, weights=['/kaggle/working/yolov5/runs/train/yolov5s_results/weights/last.pt'], imgsz=[640, 640], batch_size=1, device=cpu, half=False, inplace=False, keras=False, optimize=False, int8=False, per_tensor=False, dynamic=False, simplify=False, opset=17, verbose=False, workspace=4, nms=False, agnostic_nms=False, topk_per_class=100, topk_all=100, iou_thres=0.45, conf_thres=0.25, include=['onnx']
YOLOv5 🚀 v7.0-312-g1bcd17ee Python-3.10.13 torch-2.1.2 CPU

Fusing layers... 
custom_YOLOv5s summary: 182 layers, 7251912 parameters, 0 gradients

PyTorch: starting from /kaggle/working/yolov5/runs/train/yolov5s_results/weights/last.pt with output shape (1, 25200, 8) (14.2 MB)

ONNX: starting export with onnx 1.16.0...
ONNX: export success ✅ 1.4s, saved as /kaggle/working/yolov5/runs/train/yolov5s_results/weights/last.onnx (28.1 MB)

Export complete (2.2s)
Results saved to /kaggle/working/yolov5/runs/train/yolov5s_results/weights
Detect:          python detect.py --wei

In [24]:
!tar -czvf yolov5-b128-e100-img640.tar.gz /kaggle/working/yolov5/runs/train/yolov5s_results

tar: Removing leading `/' from member names
/kaggle/working/yolov5/runs/train/yolov5s_results/
/kaggle/working/yolov5/runs/train/yolov5s_results/events.out.tfevents.1715852419.c519e5995b40.113.0
/kaggle/working/yolov5/runs/train/yolov5s_results/P_curve.png
/kaggle/working/yolov5/runs/train/yolov5s_results/results.png
/kaggle/working/yolov5/runs/train/yolov5s_results/labels.jpg
/kaggle/working/yolov5/runs/train/yolov5s_results/train_batch0.jpg
/kaggle/working/yolov5/runs/train/yolov5s_results/results.csv
/kaggle/working/yolov5/runs/train/yolov5s_results/train_batch2.jpg
/kaggle/working/yolov5/runs/train/yolov5s_results/weights/
/kaggle/working/yolov5/runs/train/yolov5s_results/weights/last.pt
/kaggle/working/yolov5/runs/train/yolov5s_results/weights/last.onnx
/kaggle/working/yolov5/runs/train/yolov5s_results/weights/best.pt
/kaggle/working/yolov5/runs/train/yolov5s_results/weights/best.onnx
/kaggle/working/yolov5/runs/train/yolov5s_results/PR_curve.png
/kaggle/working/yolov5/runs/train/

In [ ]:
Image(filename='/kaggle/working/yolov5/runs/train/yolov5s_results/results.png', width=1000)  # view results.png

In [ ]:
Image(filename='/kaggle/working/yolov5/runs/train/yolov5s_results/val_batch0_labels.jpg', width=900)